In [1]:
import warnings
warnings.filterwarnings('ignore') # pending solution to hdf issue
warnings.simplefilter("ignore", DeprecationWarning)
import pandas as pd
pd.options.mode.chained_assignment = None # shut off SettingWithCopyWarning

# Using visualime on CMPD data

link to lime module here: https://github.com/marcotcr/lime

In [2]:
import lime.lime_tabular
import visualime
import numpy as np
%matplotlib inline

ImportError: No module named lime.lime_tabular

In [3]:
import sys
sys.version

'3.5.2 (default, Jul 17 2016, 00:00:00) \n[GCC 4.8.4]'

#### Select model_id

In [4]:
model_id = 32092

#### Retrieve model info from results.models

In [6]:
selected_model = visualime.dataframe_from_model_id(model_id)
selected_model

,model_id,model_group_id,run_time,batch_run_time,model_type,model_parameters,model_comment,batch_comment,config,test,model_hash,train_matrix_uuid,train_end_time,experiment_hash
0,32092,429,2017-03-18 05:07:01.192379,2017-03-17 22:59:12.480044,sklearn.linear_model.LogisticRegression,"{""C"": 1, ""penalty"": ""l1"", ""random_state"": 2193}",with accident as adverse,,"{""unit"": ""officer"", ""model"": [""RandomForest"", ...",False,8647537d80fcf8d60847e5bf5c9ff219,da0e5c602765c5ff83adc4874e1f9c9c,2013-04-01,None


#### Retrieve train matrix and labels

In [7]:
#MATRIX_DIR = '/mnt/data/public_safety/charlotte_eis/triage/matrix_correct_month_1y/'
MATRIX_DIR = '/mnt/data/public_safety/charlotte_eis/triage/'

In [8]:
train_matrix,train_labels = visualime.train_matrix_from_dataframe(selected_model,MATRIX_DIR)

#### Clone fitted model

In [ ]:
fitted_model = visualime.clone_model(selected_model,train_matrix,train_labels)

#### Show usable test matrices

In [10]:
possible_test_matrices = visualime.show_test_matrices(selected_model)

2014-04-01 e90eedb202234ba3a9ca8b5fcd250179
2014-04-02 1204392617c288116ae48aee9499225d
2014-04-03 3c67c797553c6c0bf1381c230579ab1b
2014-04-04 86398d85d07b8207948e8142cd3fb8f6
2014-04-05 e6720ec68cf38b617ebe90fab0810ac1
2014-04-06 9714c5c23979f4725b785cecdc288bcb
2014-04-07 a626dcde27a96d1c47e0f666bdec2b63
2014-04-08 05370f8b4c6f24d7084c052d8b8bc9b4
2014-04-09 16c4746230f6ceebba1f64923282760b
2014-04-10 78ec47c4e420c31d9f8bd9fe93cc5161
2014-04-11 725897ad93c2ce1b7e7628d2964142cf
2014-04-12 2b821b4d62b50a9c01341d6b0253402d
2014-04-13 eaf986b11eb3a0cdef2f9a6f7410aaaa
2014-04-14 ffdece815a0bf2c9e6e45bab788b5768
2014-04-15 506133659a12b5ac773185384539b318
2014-04-16 9137c3d48da0215503956b2dcad4fa1b
2014-04-17 16063fbeabbd881adb76d2a9a9b5e405
2014-04-18 be6c24a9b33ce96b23013b42d224ef95
2014-04-19 15456ae1867c3902aee3b086dd03c6bd
2014-04-20 05383683302ed848e3ad46cabbd39b4b
2014-04-21 52eab4bb1fa257dbe826e00213c7f02f
2014-04-22 7cbc89edad083f6c581d6b74bb156571
2014-04-23 dc9c6ca9496586ee9ed75

In [11]:
test_matrix_uuid = '725897ad93c2ce1b7e7628d2964142cf' # from above listing

In [12]:
test_matrix_and_labels,test_matrix = visualime.test_matrix_from_uuid(test_matrix_uuid,possible_test_matrices,MATRIX_DIR)

#### Visualize test set

In [ ]:
#n = 512
n = len(test_matrix) # number of test set points to visualize
toppct = 0.05 # percentage of scores labeled '1'

In [ ]:
plot_canvas = visualime.plot(n,toppct,fitted_model,test_matrix_and_labels,test_matrix)
plot_canvas

#### Checking std 

In [ ]:
tree_preds_all = []
for estimator in fitted_model.estimators_:
    tree_preds = [estimator.predict_proba(test_matrix.loc[i].as_matrix().reshape(1,-1))[0][0] for i in test_matrix.index]
    print(min(tree_preds),max(tree_preds))
    tree_preds_all.append(tree_preds)

#### LIME explainer object

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(train_matrix.as_matrix(), 
                                                   feature_names=list(train_matrix), 
                                                   discretize_continuous=False)

#### Nullify Large Tables

In [14]:
test_matrix_and_labels.to_csv("32092_2014-04-11_test-matrix-and-labels.csv")

In [9]:
train_matrix = None
train_labels = None
test_matrix_and_labels = None

#### Explain instance

In [ ]:
num_features = 10 # number of features for LIME to use in its model
num_samples = 5000 # size of local area
#model_regressor = 

In [ ]:
i = np.random.randint(0,test_matrix.shape[0])

In [ ]:
explainer_i = explainer.explain_instance(test_matrix.iloc[i].as_matrix(),
                                         fitted_model.predict_proba,
                                         num_features=num_features,
                                         top_labels=1,
                                         num_samples=num_samples,
                                         #model_regressor=model_regressor
                                        )

#explainer_i.show_in_notebook(show_table=True, show_all=False)

In [ ]:
explainer_i.as_list(0)